# Save Markdown text into Vector DB

## Step-1: Config

In [2]:
from my_config import MY_CONFIG

## Step-2: Read Markdown

In [3]:
import os
import glob

pattern = os.path.join(MY_CONFIG.OUTPUT_DIR_MARKDOWN, '*.md')
md_file_count = len(glob.glob(pattern, recursive=True)) 

In [4]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir=MY_CONFIG.OUTPUT_DIR_MARKDOWN, recursive=True )
documents = reader.load_data()

print (f"Loaded {len(documents)} chunks from {md_file_count} files")


Loaded 87 chunks from 87 files


In [5]:
## Inspect a sample doc
print (documents[0])

Doc ID: dfc07b1b-aa9a-4816-9b9e-c0412f6663af
Text: # Building the open future of AI  We are technology developers,
researchers, industry leaders and advocates who collaborate to advance
safe, responsible AI rooted in open innovation.  ![Conference
Speaker](https://images.prismic.io/ai-alliance/Zy08cq8jQArT0jJI_Imagef
romNotion.jpeg?auto=format%2Ccompress&fit=max&w=3840)  ![Skills &
Education](htt...


## Step-3: Setup Embedding Model

In [6]:
# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name = MY_CONFIG.EMBEDDING_MODEL
)

modules.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/60.6M [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

## Step-4: Connect to Milvus

In [8]:
# connect to vector db
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri = MY_CONFIG.DB_URI ,
    dim = MY_CONFIG.EMBEDDING_LENGTH , 
    collection_name = MY_CONFIG.COLLECTION_NAME,
    overwrite=True
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected Llama-index to Milvus instance: ", MY_CONFIG.DB_URI )

2025-02-25 14:40:41,556 [DEBUG][_create_connection]: Created new connection using: bf69c03e00f94f7aa4c5a72b362ab6c8 (async_milvus_client.py:600)


✅ Connected Llama-index to Milvus instance:  ./rag_website.db


In [9]:
## Clear up any old data

from pymilvus import MilvusClient

milvus_client = MilvusClient(MY_CONFIG.DB_URI)
print ("✅ Connected to Milvus instance: ", MY_CONFIG.DB_URI )

# if we already have a collection, clear it first
if milvus_client.has_collection(collection_name = MY_CONFIG.COLLECTION_NAME):
    milvus_client.drop_collection(collection_name = MY_CONFIG.COLLECTION_NAME)
    print ('✅ Cleared collection :', MY_CONFIG.COLLECTION_NAME)
    

✅ Connected to Milvus instance:  ./rag_website.db
✅ Cleared collection : pages


In [10]:
# connect to vector db
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri = MY_CONFIG.DB_URI ,
    dim = MY_CONFIG.EMBEDDING_LENGTH , 
    collection_name = MY_CONFIG.COLLECTION_NAME,
    overwrite=True
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected Llama-index to Milvus instance: ", MY_CONFIG.DB_URI )

2025-02-25 14:40:42,708 [DEBUG][_create_connection]: Created new connection using: 74d5f252a97c4989bf5ce1870a2c2d80 (async_milvus_client.py:600)


✅ Connected Llama-index to Milvus instance:  ./rag_website.db


## Step-5: Save to DB

In [11]:
%%time

# create an index

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
print ("✅ Saved documents to db ", MY_CONFIG.DB_URI )

✅ Saved documents to db  ./rag_website.db
CPU times: user 975 ms, sys: 168 ms, total: 1.14 s
Wall time: 2.4 s
